<a href="https://colab.research.google.com/github/rayruchira/UT-CML/blob/main/Distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning DistilBERT for MultiLabel Text Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00


In [ ]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
data = pd.read_csv('/content/Cleaned Allison First 50.csv', )
data=data[0:49]
print(data.iloc[48])

Item Type                                                                              journalArticle
Publication Year                                                                               2021.0
Author                                                                                 Murthy, Dhiraj
Title                                               Evaluating Platform Accountability: Terrorist ...
Publication Title                                                       American Behavioral Scientist
DOI                                                                                               NaN
Url                                                 https://ezproxy.lib.utexas.edu/login?url=https...
Abstract Note                                       YouTube has traditionally been singled out as ...
Keywords                                            social media; social network analysis; algorit...
Database                                                                          

In [ ]:
data.drop(['Item Type', 'Publication Year', 'Author', 'Publication Title', 'DOI', 'Url', 'Database', 'Citation'], inplace=True, axis=1)
# data.drop(['id'], inplace=True, axis=1)
print(data.iloc[48])

Title                                               Evaluating Platform Accountability: Terrorist ...
Abstract Note                                       YouTube has traditionally been singled out as ...
Keywords                                            social media; social network analysis; algorit...
Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                                x
Mis/Disinformation                                                                                  x
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                             

In [ ]:
data["text"] = data['Title'] +" "+ data["Abstract Note"] + " "+ data["Keywords"]
print(data.iloc[0])


Title                                               #TurntTrending: a systematic review of substan...
Abstract Note                                       Aims: There is a growing body of literature ex...
Keywords                                            Social media; Sentiment analysis; Public opini...
Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                              x
Other                                                                             

In [ ]:
data.drop(['Title', 'Abstract Note', 'Keywords'], inplace=True, axis=1)
print(data.iloc[0])

Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                              x
Other                                                                                             NaN
Text                                                #TurntTrending: a systematic review of substan...
Name: 0, dtype: object


In [ ]:
cols = ['Social Movements, politics, policy, and Activism', 'News', "Mis/Disinformation", "Social Stratification or Segregation", "Economic sociology", "Health", "Other"]
data['labels']= [[1 if e==e else 0 for e in row ] for row in data[cols].values.tolist()]

In [ ]:
data.iloc[48]

Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                                x
Mis/Disinformation                                                                                  x
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                                             NaN
Text                                                Evaluating Platform Accountability: Terrorist ...
labels                                                                          [0, 1, 1, 0, 0, 0, 0]
Name: 48, dtype: object

In [ ]:
data.drop(cols, inplace=True, axis=1)


In [ ]:
data.iloc[48]

Text      Evaluating Platform Accountability: Terrorist ...
labels                                [0, 1, 1, 0, 0, 0, 0]
Name: 48, dtype: object

In [ ]:
# data1 = pd.read_csv( "/content/cleaned mine2.csv" )
# # print(data1)
# data2=data1[0:96]
# print(data2)
# data3=data1[1805:]
# data3=data3[:-1]
# print(data3)
# frames = [data2, data3]
# data1 = pd.concat(frames)
# print(data1)


data1 = pd.read_csv( "/content/cleaned.csv" )
data1 = data1[143:1805]
# data2=data1[1:96]
# print(data2)
# data3=data1[1805:]
# data3=data3[:-1]
# print(data3)
# frames = [data2, data3]
# data1 = pd.concat(frames)
print(data1)

           Item Type  Publication Year  \
143   journalArticle            2016.0   
144   journalArticle            2013.0   
145   journalArticle            2019.0   
146   journalArticle            2019.0   
147   journalArticle            2019.0   
...              ...               ...   
1800  journalArticle            2021.0   
1801  journalArticle            2020.0   
1802  journalArticle            2020.0   
1803  journalArticle            2018.0   
1804  journalArticle            2017.0   

                                                 Author  \
143   Shah, Dhavan V.; Hanna, Alex; Bucy, Erik P.; L...   
144   Aday, Sean; Farrell, Henry; Freelon, Deen; Lyn...   
145   Saffer, Adam J.; Yang, Aimei; Morehouse, Jorda...   
146      Golan, Guy J.; Manor, Ilan; Arceneaux, Phillip   
147                                      Sam, Cecile H.   
...                                                 ...   
1800                            Harju, AA; Huhtamaki, J   
1801                   

In [ ]:
data1.drop(['Item Type', 'Publication Year', 'Author', 'Publication Title', 'DOI', 'Url', 'Database', 'Citation'], inplace=True, axis=1)
# data.drop(['id'], inplace=True, axis=1)

# Use del keywords
data1.pop(data1.columns[-1])

print(data1.iloc[96])

Title                                               You truly are the worst kind of racist!': Argu...
Abstract Note                                       The role of women in populist and radical righ...
Keywords                                            Practical politics; Social media; Twitter (Web...
Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                             

In [ ]:
data1["text"] = data1['Title'].fillna("") +" "+ data1["Abstract Note"].fillna("") + " "+data1["Keywords"].fillna("")
print(data1.iloc[0])


Title                                                     Dual Screening During Presidential Debates.
Abstract Note                                       The impact of presidential debates on candidat...
Keywords                                            Practical politics; Political participation; s...
Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                             

In [ ]:
data1.drop(['Title', 'Abstract Note', 'Keywords'], inplace=True, axis=1)
print(data1.iloc[0])

Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                                             NaN
text                                                Dual Screening During Presidential Debates. Th...
Name: 143, dtype: object


In [ ]:
cols = ['Social Movements, politics, policy, and Activism', 'News', "Mis/Disinformation", "Social Stratification or Segregation", "Economic sociology", "Health", "Other"]
data1['labels']= [[1 if e==e else 0 for e in row ] for row in data1[cols].values.tolist()]

In [ ]:
data1.iloc[0]

Social Movements, politics, policy, and Activism                                                  NaN
News                                                                                              NaN
Mis/Disinformation                                                                                NaN
Social Stratification or Segregation                                                              NaN
Economic sociology                                                                                NaN
Health                                                                                            NaN
Other                                                                                             NaN
text                                                Dual Screening During Presidential Debates. Th...
labels                                                                          [0, 0, 0, 0, 0, 0, 0]
Name: 143, dtype: object

In [ ]:
data1.drop(cols, inplace=True, axis=1)


In [ ]:
data1.iloc[96]

text      You truly are the worst kind of racist!': Argu...
labels                                [0, 0, 0, 0, 0, 0, 0]
Name: 239, dtype: object

In [ ]:
new_df = data1

In [ ]:
# new_df= data.rename(columns={"Text": "text"})
frames = [new_df, data1]
new_df = pd.concat(frames)

In [ ]:
new_df

,text,labels
0,#TurntTrending: a systematic review of substan...,"[0, 0, 0, 0, 0, 1, 0]"
1,A message from Addiction's new Editor‐in‐Chief...,"[0, 0, 0, 0, 0, 0, 1]"
2,Action is needed to deter the trumpeting of no...,"[0, 0, 0, 0, 0, 1, 0]"
3,An alcohol and other drugs library: building c...,"[0, 1, 0, 0, 0, 0, 0]"
4,Opioid use stigmatization and destigmatization...,"[0, 0, 0, 0, 0, 1, 0]"
...,...,...
2050,The populist body in the age of social media: ...,"[1, 1, 0, 1, 0, 0, 0]"
2051,Workblogging in a Facebook age In keeping with...,"[0, 0, 0, 0, 0, 0, 1]"
2052,Controlling the Flow: How Urban Music Videos A...,"[1, 1, 0, 1, 0, 0, 0]"
2053,From Youthful Experimentation to Professional ...,"[0, 0, 1, 0, 0, 0, 1]"


<a id='section03'></a>
### Preparing the Dataset and Dataloader

#### *MultiLabelDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT model for training. 
- We are using the DistilBERT tokenizer to tokenize the data in the `text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids`

- `targets` is the list of categories labled as `0` or `1` in the dataframe. 
- The *MultiLabelDataset* class is used to create 2 datasets, for training and for validation.


In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 500
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=False, do_lower_case=True)

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'text': text,
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.95
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

# training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (1662, 2)
TRAIN Dataset: (0, 2)
TEST Dataset: (1662, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

# training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `DistilBERTClass`. 
 - This network will have the `DistilBERT` model.  Follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively. 
 - In the forward loop, there are 2 output from the `DistilBERTClass` layer.
 - The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`. 
 - Keep note the number of dimensions for `Linear Layer` is **6** because that is the total number of categories in which we are looking to classify our model.
 - The data will be fed to the `DistilBERTClass` as defined in the dataset. 
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 7)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

#### Train

In [ ]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2it [00:00, 15.00it/s]

Epoch: 0, Loss:  0.7107506394386292


94it [00:05, 15.92it/s]
4it [00:00, 17.13it/s]

Epoch: 1, Loss:  0.5833714604377747


94it [00:05, 15.72it/s]
2it [00:00, 15.69it/s]

...

94it [00:05, 15.94it/s]
4it [00:00, 17.59it/s]

Epoch: 498, Loss:  0.0004997117794118822


94it [00:05, 16.46it/s]
3it [00:00, 19.07it/s]

Epoch: 499, Loss:  0.00042688887333497405


94it [00:06, 15.46it/s]



### Validating the Model




In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    fin_ids=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            text=data['text']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            fin_ids.extend(text)
    return fin_outputs, fin_targets, fin_ids

In [ ]:
outputs, targets, ids= validation(testing_loader)


final_outputs = np.array(outputs) >=0.5



416it [00:15, 27.47it/s]


In [ ]:
print(ids)
print(targets)
print(final_outputs)

['Dual Screening During Presidential Debates. The impact of presidential debates on candidate evaluations remains an open topic. Research has long sought to identify the factors that matter most in citizens’ responses to debate content, including what candidates say, how they say it, and the manner in which they appear. This study uses detailed codings of the first and third 2012 presidential debates to evaluate the impact of candidates’ verbal and nonverbal behaviors on viewers’ “second screen” response—their use of computers, tablets, and mobile devices to express their reactions to the viewing experience. To examine the relationship between candidates’ on-screen behaviors and the social media response, we conduct generalized least squares regression (Prais–Winstein estimation) relating two data sources: (a) a shot-by-shot content analysis coded for rhetorical/functional, tonal, and visual elements of both candidates’ behavior during the debates, and (b) corresponding real-time measu

In [ ]:
df = pd.DataFrame(final_outputs, columns=cols)

In [ ]:
df.to_csv(r'tags.csv', index=False)

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.41025641025641024
Hamming Loss = 0.21978021978021978



### Saving the Trained Model for inference


In [ ]:
# Saving the files for inference

output_model_file = '/content/drive/MyDrive/model weights/pytorch_distilbert_news.bin'
output_vocab_file = '/content/drive/MyDrive/model weights/vocab_distilbert_news.bin'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

Saved


In [ ]:
model=torch.load('/content/drive/MyDrive/model weights/pytorch_distilbert_news.bin')